# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770878960014                   -0.52    9.0         
  2   -2.772147987870       -2.90       -1.32    1.0    139ms
  3   -2.772170376554       -4.65       -2.47    1.0    141ms
  4   -2.772170651376       -6.56       -3.17    1.0    158ms
  5   -2.772170721577       -7.15       -3.90    2.0    163ms
  6   -2.772170722689       -8.95       -4.30    1.0    164ms
  7   -2.772170723011       -9.49       -5.47    1.0    148ms
  8   -2.772170723015      -11.44       -6.27    2.0    171ms
  9   -2.772170723015      -13.29       -6.55    1.0    170ms
 10   -2.772170723015      -14.65       -7.25    1.0    164ms
 11   -2.772170723015      -13.72       -7.96    2.0    203ms
 12   -2.772170723015   +  -13.88       -8.35    1.0    155ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.773558038778103

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770765952358                   -0.52    8.0         
  2   -2.772056813709       -2.89       -1.32    1.0    184ms
  3   -2.772083088837       -4.58       -2.50    1.0    142ms
  4   -2.772083362218       -6.56       -3.27    1.0    141ms
  5   -2.772083415303       -7.28       -3.83    2.0    186ms
  6   -2.772083417754       -8.61       -5.18    1.0    146ms
  7   -2.772083417808      -10.27       -5.27    2.0    181ms
  8   -2.772083417811      -11.54       -6.58    1.0    152ms
  9   -2.772083417811      -13.05       -7.08    2.0    179ms
 10   -2.772083417811   +  -14.51       -7.05    1.0    160ms
 11   -2.772083417811   +  -13.95       -8.68    1.0    163ms

Polarizability via ForwardDiff:       1.7725349794882892
Polarizability via finite difference: 1.773558038778103
